In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import OpenAIChat
from langchain.prompts import PromptTemplate

import chromadb
import os

DB_PATH = 'chromadb4'

In [2]:
if os.path.exists(f'{DB_PATH}/'):
    db = Chroma(persist_directory=DB_PATH, embedding_function=OpenAIEmbeddings())
else:
    raw_documents = PyPDFDirectoryLoader('assets/').load()
    text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    documents = text_splitter.split_documents(raw_documents)
    db = Chroma.from_documents(documents, embedding_function = OpenAIEmbeddings(), persist_directory=DB_PATH)

In [13]:
SEARCH_PROMPT = """
Im folgenden bekommst du mehrere Dokumente: <DOKUMENT1>, <DOKUMENT2> und <DOKUMENT3>.
Der Nutzer ist auf der Suche nach einem Rezept, welches du in einem der drei Dokumente findest.
Bestimme zunächst das passenste Rezept für die <REZEPTANFRAGE> des Nutzers.

Gib dann das Rezept in komprimerter Form zurück.
Dabei darfst du für drei Sätze in dem Grundrezept maximal zwei Sätze verwenden.
Achte darauf, dass du Mengenangaben und Temperaturen innerhalb der Schritte angibst.

Ausgabeformat:
```
<REZEPTNAME>

Zutaten:
- <ZUTAT1>
- <ZUTAT2>
- <ZUTAT3>

Schritt 1: <SCHRITT1>
Schritt 2: <SCHRITT2>
Schritt 3: <SCHRITT3>
...

Quelle: <QUELLE>
```

<DOKUMENT1>: {DOKUMENT1}
<QUELLE1>: {QUELLE1}
<DOKUMENT2>: {DOKUMENT2}
<QUELLE2>: {QUELLE2}
<DOKUMENT3>: {DOKUMENT3}
<QUELLE3>: {QUELLE3}
<REZEPTANFRAGE>: {REZEPTANFRAGE}

"""

In [18]:
query = "Pizzateig"
docs = db.similarity_search(query)
most_likely_document = docs[0].page_content

In [19]:
p_template = PromptTemplate.from_template(SEARCH_PROMPT)

In [20]:
chat = OpenAIChat(temperature=0)
print(chat.predict(p_template.format(
    DOKUMENT1 = docs[0].page_content,
    QUELLE1 = docs[0].metadata['source'],
    DOKUMENT2 = docs[1].page_content,
    QUELLE2 = docs[1].metadata['source'],
    DOKUMENT3 = docs[2].page_content,
    QUELLE3 = docs[2].metadata['source'],
    REZEPTANFRAGE = query)))

/Users/hilko/Documents/Git/langchain-talk2docs/venv/lib/python3.10/site-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Grundrezept für Pizzateig

Zutaten:
- 1 kg Weizenmehl (Type 550 oder eine Mischung aus einem Drittel 1050 und zwei Dritteln 405)
- 20 g Hefe (1/2 Würfel)
- ca. 750 ml lauwarmes Wasser
- 1 gehäufter TL Salz
- 2-3 EL Olivenöl

Schritt 1: Das Mehl in die Rührschüssel der Küchenmaschine füllen. Die Hefe zerbröckeln und in einer Tasse lauwarmen Wassers auflösen. Die Maschine einschalten und langsam die Flüssigkeit hinzufließen lassen. Danach so viel wie nötig vom restlichen Wasser hinzugeben, sodass ein weicher Teig entsteht.
Schritt 2: Dann natürlich auch Salz und Öl zufügen. Am besten Olivenöl, das gibt nochmal mehr Geschmack. Die Maschine zuerst auf langsamer Stufe etwa fünf Minuten arbeiten lassen, bis sich der Teig glatt vom Schüsselrand löst. Danach aber noch mindestens weitere drei Minuten kneten lassen.
Schritt 3: Nun mit bemehlten Händen aus der Rührschüssel heben und in eine mit Mehl ausgestäubte Schüssel geben. Am Ende eventuell nochmal mit ein wenig Mehl bestäuben und dann mit e